# 1. 강의영상 

# 2. Imports 

In [1]:
import transformers
import datasets
import huggingface_hub
import torch
import torchvision
import pytorchvideo.data
import PIL
import tarfile
import mp2024pkg as mp

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. Model 입력

`-` 아래중 하나의 방법순으로.. 

- 방법1: `model.forward?` 에서 시그니처를 확인
- 방법2: `model.forward?` 에서 사용예제를 확인
- 방법3: 인터넷을 활용한 외부 자료 확인 (공식문서, 공식튜토리얼, 신뢰할만한 블로그, ChatGPT등)
- 방법4: `model.forward??` 를 보고 모든 코드를 뜯어봄 <--- 하지마세요

`-` 모델의 입력이 어떤형태로 정리되어야 하는지 알아내는 확실한 방법은 없음 

- 방법1,2,3 은 다른사람의 호의에 기대해야함.
- 방법4는 사실상 불가능

`# 예제1` -- 텍스트분류

In [2]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [3]:
model1.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

- 최대단어수 = 512

*모델의 입력파악*

In [4]:
model1.forward?

Signature:
model1.forward(
    input_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    labels: Optional[torch.LongTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[transformers.modeling_outputs.SequenceClassifierOutput, Tuple[torch.Tensor, ...]]
Docstring:
The [`DistilBertForSequenceClassification`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
        Indices of input sequence 

*사용예시1 -- 입력나열, loss O*

In [5]:
model1.forward(
    input_ids = torch.tensor([[1,2,3],[2,3,4]]),
    labels = torch.tensor([1,0])
)

SequenceClassifierOutput(loss=tensor(0.6945, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0012,  0.0777],
        [-0.0101,  0.0711]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [6]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3],[2,3,4]]),
    labels = torch.tensor([1,0])
)
model1.forward(**model1_input)

SequenceClassifierOutput(loss=tensor(0.6945, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0012,  0.0777],
        [-0.0101,  0.0711]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [7]:
model1(
    input_ids = torch.tensor([[1,2,3],[2,3,4]])
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0012,  0.0777],
        [-0.0101,  0.0711]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [8]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3],[2,3,4]]),
)
model1(**model1_input)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0012,  0.0777],
        [-0.0101,  0.0711]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [9]:
model1(
    torch.tensor([[1,2,3],[2,3,4]])
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0012,  0.0777],
        [-0.0101,  0.0711]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

> 사용예시1~5에서 `model1()` 대신에 `model1.forward()`를 사용해도 된다. 

`#`

`# 예제2` -- 이미지분류

In [10]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [11]:
model2.config

ViTConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.2"
}

- 이미지크기: 224

*모델의 입력파악*

In [ ]:
model2.forward?

Signature:
model2.forward(
    pixel_values: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    labels: Optional[torch.Tensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    interpolate_pos_encoding: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[tuple, transformers.modeling_outputs.ImageClassifierOutput]
Docstring:
The [`ViTForImageClassification`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    pixel_values (`torch.FloatTensor` of shape `(batch_size, num_channels, height, width)`):
        Pixel values. Pixel values can be obtained using [`AutoImageProcessor`]. See [`ViTImageProc

*사용예시1 -- 입력나열, loss O*

In [13]:
model2(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,0])
)

ImageClassifierOutput(loss=tensor(1.1209, grad_fn=<NllLossBackward0>), logits=tensor([[0.0642, 0.1145, 0.0695],
        [0.0579, 0.1165, 0.0758]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [14]:
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,0])
)
model2(**model2_input)

ImageClassifierOutput(loss=tensor(1.1137, grad_fn=<NllLossBackward0>), logits=tensor([[0.0601, 0.1022, 0.0659],
        [0.0666, 0.1041, 0.0707]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [15]:
model2(
    pixel_values = torch.randn(2,3,224,224),
)

ImageClassifierOutput(loss=None, logits=tensor([[0.0973, 0.1043, 0.0674],
        [0.0890, 0.1302, 0.0539]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [16]:
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
)
model2(**model2_input)

ImageClassifierOutput(loss=None, logits=tensor([[0.0769, 0.1314, 0.0494],
        [0.0713, 0.0917, 0.0259]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [17]:
model2(
    torch.randn(2,3,224,224),
)

ImageClassifierOutput(loss=None, logits=tensor([[0.0644, 0.1262, 0.0855],
        [0.0257, 0.1343, 0.0867]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제3` -- 동영상분류

In [18]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [19]:
model3.config

VideoMAEConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "MCG-NJU/videomae-base",
  "architectures": [
    "VideoMAEForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 16,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.2",
  "tubelet_size": 2,
  "use_mean_pooling": false
}

- 이미지크기: 224
- 프레임: 16

*모델의 입력파악*

In [20]:
model3.forward?

Signature:
model3.forward(
    pixel_values: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    labels: Optional[torch.Tensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, transformers.modeling_outputs.ImageClassifierOutput]
Docstring:
The [`VideoMAEForVideoClassification`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    pixel_values (`torch.FloatTensor` of shape `(batch_size, num_frames, num_channels, height, width)`):
        Pixel values. Pixel values can be obtained using [`AutoImageProcessor`]. See
        [`VideoMAEImageProcessor.__call__`] for de

*사용예시1 -- 입력나열, loss O*

In [21]:
model3(
    pixel_values = torch.randn(2,16,3,224,224),
    labels = torch.tensor([1,0])
)

ImageClassifierOutput(loss=tensor(0.8626, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1358,  0.3941],
        [-0.2414,  0.6883]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [22]:
model3_input = dict(
    pixel_values = torch.randn(2,16,3,224,224),
    labels = torch.tensor([1,0])
)
model3(**model3_input)

ImageClassifierOutput(loss=tensor(0.6844, grad_fn=<NllLossBackward0>), logits=tensor([[-0.2486,  0.5740],
        [-0.1603,  0.3884]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [23]:
model3(
    pixel_values = torch.randn(2,16,3,224,224),
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.0368,  0.5111],
        [-0.0464,  0.4651]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [24]:
model3_input = dict(
    pixel_values = torch.randn(2,16,3,224,224),
)
model3(**model3_input)

ImageClassifierOutput(loss=None, logits=tensor([[-0.2656,  0.5812],
        [-0.1089,  0.5978]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [25]:
model3(
    torch.randn(2,16,3,224,224),
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.2789,  0.5706],
        [-0.2000,  0.6558]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

# 4. Model 사용 연습 

## A. 텍스트

In [3]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- imdb

In [4]:
imdb = datasets.load_dataset('imdb')
d = imdb['train'].select(range(3))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

`(풀이1)`

*실패*

In [10]:
model1(
    input_ids = torch.tensor(tokenizer(d['text'])['input_ids']),
    labels = torch.tensor([0,0,1])
)

ValueError: expected sequence of length 363 at dim 1 (got 304)

*원인분석*

In [15]:
mp.show_list(tokenizer(d['text'])['input_ids'])

Level 1 - Type: list, Length: 3, Content: [[101, 1045, 12524, 1045, ... // ... , 7987, 1013, 1028, 102]]
     Level 2 - Type: list, Length: 363, Content: [101, 1045, 12524, 1045,  ... // ... 7, 1037, 5436, 1012, 102]
     Level 2 - Type: list, Length: 304, Content: [101, 1000, 1045, 2572, 8 ... // ... 5, 1055, 4230, 1012, 102]
     Level 2 - Type: list, Length: 133, Content: [101, 2065, 2069, 2000, 4 ... // ... 6, 7987, 1013, 1028, 102]


In [12]:
mp.show_list(
    tokenizer(d['text'],padding=True)['input_ids']
)

Level 1 - Type: list, Length: 3, Content: [[101, 1045, 12524, 1045, ... // ...  0, 0, 0, 0, 0, 0, 0, 0]]
     Level 2 - Type: list, Length: 363, Content: [101, 1045, 12524, 1045,  ... // ... 7, 1037, 5436, 1012, 102]
     Level 2 - Type: list, Length: 363, Content: [101, 1000, 1045, 2572, 8 ... // ... , 0, 0, 0, 0, 0, 0, 0, 0]
     Level 2 - Type: list, Length: 363, Content: [101, 2065, 2069, 2000, 4 ... // ... , 0, 0, 0, 0, 0, 0, 0, 0]


*성공*

In [13]:
model1(
    input_ids = torch.tensor(tokenizer(d['text'],padding=True)['input_ids']),
    labels = torch.tensor([0,0,1])
)

SequenceClassifierOutput(loss=tensor(0.7086, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0432,  0.0316],
        [-0.0439,  0.0662],
        [-0.0688,  0.0301]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`(풀이2)`

In [16]:
tokenizer(d['text'],padding=True,return_tensors='pt')['input_ids']

tensor([[  101,  1045, 12524,  ...,  5436,  1012,   102],
        [  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  2065,  2069,  ...,     0,     0,     0]])

In [17]:
model1(
    input_ids = tokenizer(d['text'],padding=True,return_tensors='pt')['input_ids'],
    labels = torch.tensor([0,0,1])
)

SequenceClassifierOutput(loss=tensor(0.7086, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0432,  0.0316],
        [-0.0439,  0.0662],
        [-0.0688,  0.0301]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [19]:
model1(
    tokenizer(d['text'],padding=True,return_tensors='pt')['input_ids'],
    labels = torch.tensor([0,0,1])
)

SequenceClassifierOutput(loss=tensor(0.7086, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0432,  0.0316],
        [-0.0439,  0.0662],
        [-0.0688,  0.0301]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제2` -- emotion

In [20]:
emotion = datasets.load_dataset('emotion')

In [21]:
d = emotion['train'].select(range(3))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

`(풀이)`

In [23]:
model1(
    torch.tensor(tokenizer(d['text'],padding=True)['input_ids'])
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0105,  0.0449],
        [ 0.0015,  0.0298],
        [-0.0091,  0.0532]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
model1(
    tokenizer(d['text'],padding=True,return_tensors="pt")['input_ids']
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0105,  0.0449],
        [ 0.0015,  0.0298],
        [-0.0091,  0.0532]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제3` -- MBTI

In [25]:
d = datasets.Dataset.from_csv("mbti_1.csv").select(range(3))
d

Dataset({
    features: ['type', 'posts'],
    num_rows: 3
})

`(풀이1)`

*실패*

In [228]:
model1(
    tokenizer(d['posts'],padding=True,return_tensors="pt")['input_ids']
)

RuntimeError: The size of tensor a (2102) must match the size of tensor b (512) at non-singleton dimension 1

*원인분석*

In [27]:
mp.show_list(tokenizer(d['posts'],padding=True,return_tensors="pt")['input_ids'])

Token indices sequence length is longer than the specified maximum sequence length for this model (2102 > 512). Running this sequence through the model will result in indexing errors


Level 1 - Type: Tensor, Length: 3, Content: tensor([[ 101, 1005, 8299 ... // ...  ...,    0,    0,    0]])
     Level 2 - Type: Tensor, Length: 2102, Content: tensor([ 101, 1005, 8299,  ...,    0,    0,    0])
     Level 2 - Type: Tensor, Length: 2102, Content: tensor([ 101, 1005, 1045,  ..., 1012, 1005,  102])
     Level 2 - Type: Tensor, Length: 2102, Content: tensor([ 101, 1005, 2204,  ...,    0,    0,    0])


In [31]:
mp.show_list(tokenizer(d['posts'],truncation=True,return_tensors="pt")['input_ids'])

Level 1 - Type: Tensor, Length: 3, Content: tensor([[ 101, 1005, 8299 ... // ...  ..., 1012, 2077,  102]])
     Level 2 - Type: Tensor, Length: 512, Content: tensor([  101,  1005,  82 ... // ... 7,
         2215,   102])
     Level 2 - Type: Tensor, Length: 512, Content: tensor([  101,  1005,  10 ... // ... 9,
         2028,   102])
     Level 2 - Type: Tensor, Length: 512, Content: tensor([  101,  1005,  22 ... // ... 2,
         2077,   102])


*성공*

In [32]:
model1(
    tokenizer(d['posts'],truncation=True,return_tensors="pt")['input_ids']
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0469, -0.0175],
        [-0.0534, -0.0048],
        [-0.0759, -0.0220]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`(풀이2)` *--모델설정변경 (퀴즈5, 모델의 프레임수를 4로 바꾸는 예제에서 사용한 테크닉)*

*distilbert/distilbert-base-uncased 설정값 부르기*

In [35]:
config = transformers.AutoConfig.from_pretrained("distilbert/distilbert-base-uncased")
config

DistilBertConfig {
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

*설정값체크 -- `config.` + tab*

In [60]:
mp.tab(config)

*설정값변경*

In [61]:
config.max_position_embeddings = 4096
config.num_labels = 2 # 이건 잘 설정되어있어서.. 

*설정값으로 모델불러오기*

In [62]:
model1_large = transformers.AutoModelForSequenceClassification.from_config(config)

*모델사용*

In [63]:
model1_large(
    tokenizer(d['posts'],padding=True,return_tensors="pt")['input_ids']
)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1689, -0.3778],
        [ 0.0231, -0.3971],
        [-0.2340, -0.2929]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제4` -- sms_spam

In [64]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})

In [65]:
d = sms_spam['train'].select(range(3))
d

Dataset({
    features: ['sms', 'label'],
    num_rows: 3
})

`(풀이)`

In [67]:
model1(
    tokenizer(d['sms'],padding=True,return_tensors="pt")['input_ids']
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0414, -0.0771],
        [-0.0181,  0.0315],
        [ 0.0169,  0.0101]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

## B. 이미지

In [76]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- food101

In [77]:
d = datasets.load_dataset("food101", split="train[:3]")
d

Dataset({
    features: ['image', 'label'],
    num_rows: 3
})

`(예비학습)` -- `torchvision.transforms` 에서 제공하는 기능들은 배치처리가 가능한가? 

In [85]:
totensor = torchvision.transforms.ToTensor()

In [89]:
totensor(d['image'][0]) # 하나의 obs는 처리가능 

tensor([[[0.1216, 0.1137, 0.1098,  ..., 0.0039, 0.0039, 0.0000],
         [0.1255, 0.1216, 0.1176,  ..., 0.0039, 0.0039, 0.0000],
         [0.1294, 0.1255, 0.1255,  ..., 0.0039, 0.0000, 0.0000],
         ...,
         [0.2588, 0.2745, 0.2863,  ..., 0.3765, 0.3882, 0.3922],
         [0.2353, 0.2471, 0.2667,  ..., 0.3373, 0.3373, 0.3373],
         [0.2235, 0.2275, 0.2471,  ..., 0.3333, 0.3176, 0.3059]],

        [[0.1373, 0.1294, 0.1255,  ..., 0.1020, 0.1020, 0.0980],
         [0.1412, 0.1373, 0.1333,  ..., 0.1020, 0.1020, 0.0980],
         [0.1451, 0.1412, 0.1412,  ..., 0.1020, 0.0980, 0.0980],
         ...,
         [0.2471, 0.2627, 0.2745,  ..., 0.3647, 0.3765, 0.3882],
         [0.2235, 0.2353, 0.2549,  ..., 0.3255, 0.3333, 0.3333],
         [0.2118, 0.2157, 0.2353,  ..., 0.3216, 0.3137, 0.3020]],

        [[0.1412, 0.1333, 0.1294,  ..., 0.0902, 0.0902, 0.0863],
         [0.1451, 0.1412, 0.1451,  ..., 0.0902, 0.0902, 0.0863],
         [0.1490, 0.1451, 0.1529,  ..., 0.0902, 0.0863, 0.

In [91]:
totensor(d['image']) # 배치는 처리불가능

TypeError: pic should be PIL Image or ndarray. Got <class 'list'>

`(풀이)`

In [95]:
compose = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224,224)) 
])

In [96]:
compose(d['image'][0])

tensor([[[0.1223, 0.1166, 0.1159,  ..., 0.0054, 0.0041, 0.0014],
         [0.1314, 0.1290, 0.1266,  ..., 0.0037, 0.0024, 0.0033],
         [0.1425, 0.1380, 0.1339,  ..., 0.0040, 0.0026, 0.0009],
         ...,
         [0.2242, 0.2150, 0.2476,  ..., 0.3578, 0.3704, 0.3554],
         [0.2610, 0.2583, 0.2324,  ..., 0.3488, 0.3618, 0.3625],
         [0.2408, 0.2613, 0.2533,  ..., 0.3370, 0.3427, 0.3388]],

        [[0.1380, 0.1323, 0.1316,  ..., 0.1040, 0.1022, 0.0995],
         [0.1471, 0.1447, 0.1423,  ..., 0.1018, 0.0985, 0.0935],
         [0.1582, 0.1537, 0.1495,  ..., 0.1001, 0.0921, 0.0865],
         ...,
         [0.2155, 0.2087, 0.2431,  ..., 0.3306, 0.3478, 0.3367],
         [0.2492, 0.2468, 0.2231,  ..., 0.3224, 0.3469, 0.3519],
         [0.2290, 0.2496, 0.2419,  ..., 0.3107, 0.3303, 0.3340]],

        [[0.1421, 0.1409, 0.1434,  ..., 0.0922, 0.0904, 0.0877],
         [0.1565, 0.1559, 0.1541,  ..., 0.0900, 0.0874, 0.0844],
         [0.1700, 0.1654, 0.1614,  ..., 0.0890, 0.0832, 0.

In [97]:
model2(
    pixel_values = torch.stack(list(map(compose,d['image']))),
    labels = torch.tensor([0,1,0]) # 그냥 확인없이 아무거나 넣음
)

ImageClassifierOutput(loss=tensor(1.1654, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0286, -0.1753, -0.0153],
        [ 0.0542, -0.2082,  0.0403],
        [-0.0875, -0.1001,  0.1060]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제2`

In [98]:
beans = datasets.load_dataset('beans')
d = beans['train'].select(range(3))
d

Dataset({
    features: ['image_file_path', 'image', 'labels'],
    num_rows: 3
})

`(풀이)`

In [363]:
model2(
    torch.stack(list(map(compose,d['image'])))
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.2100, -0.1126, -0.1297],
        [-0.1711, -0.0252, -0.0553],
        [-0.1465,  0.0183, -0.0034]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

## C. 동영상 

In [100]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- UCF101_subset

In [7]:
file_path = huggingface_hub.hf_hub_download(
    repo_id="sayakpaul/ucf101-subset",
    filename="UCF101_subset.tar.gz",
    repo_type="dataset"
)
# file_path는 다운로드한 압축파일이 존재하는 경로와 파일명이 string으로 저장되어있음.
with tarfile.open(file_path) as t:
     t.extractall("./data") # 여기에서 "."은 현재폴더라는 의미

In [8]:
mp.tree("./data/UCF101_subset")

├── test
│   ├── ApplyEyeMakeup
│   │   ├── UCF101
│   │   ├── v_ApplyEyeMakeup_g03_c01.avi
│   │   └── ...
│   │   └── v_ApplyEyeMakeup_g23_c06.avi
│   ├── ApplyLipstick
│   │   ├── UCF101
│   │   ├── v_ApplyLipstick_g14_c01.avi
│   │   └── ...
│   │   └── v_ApplyLipstick_g16_c04.avi
│   └── ...
│   └── BenchPress
│       ├── UCF101
│       ├── v_BenchPress_g05_c02.avi
│       └── ...
│       └── v_BenchPress_g25_c06.avi
├── train
│   ├── ApplyEyeMakeup
│   │   ├── UCF101
│   │   ├── v_ApplyEyeMakeup_g02_c03.avi
│   │   └── ...
│   │   └── v_ApplyEyeMakeup_g25_c07.avi
│   ├── ApplyLipstick
│   │   ├── UCF101
│   │   ├── v_ApplyLipstick_g01_c02.avi
│   │   └── ...
│   │   └── v_ApplyLipstick_g24_c05.avi
│   └── ...
│   └── BenchPress
│       ├── UCF101
│       ├── v_BenchPress_g01_c05.avi
│       └── ...
│       └── v_BenchPress_g24_c05.avi
└── val
    ├── ApplyEyeMakeup
    │   ├── UCF101
    │   ├── v_ApplyEyeMakeup_g01_c01.avi
    │   ├── v_ApplyEyeMakeup_g14_c05.avi
    │   └── v_A

In [101]:
video_path = "./data/UCF101_subset/test/BenchPress/v_BenchPress_g05_c02.avi"
video = pytorchvideo.data.encoded_video.EncodedVideo.from_path(video_path).get_clip(0, float('inf'))['video']
video.shape

torch.Size([3, 67, 240, 320])

`(풀이)`

In [102]:
video.permute(1,0,2,3)[:16,:,:224,:224].shape

torch.Size([16, 3, 224, 224])

In [108]:
model3(
    #video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    #video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.1593, -0.2056]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)